In [33]:
prop = pd.read_csv("properties_2016.csv")
train = pd.read_csv("train_2016_v2.csv")
train2016 = train.merge(prop, how="left", on = 'parcelid')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
train2016 = df_train

In [34]:
prop_garages =  ['garagecarcnt', 'garagetotalsqft']
prop_yards = ['yardbuildingsqft17']
prop_pools = ['poolcnt', 'poolsizesum','pooltypeid10', 'pooltypeid2', 'pooltypeid7']

#  COLUMNS TO DROP
'yardbuildingsqft26' (데이터의 개수 96개), 'hashottuborspa' (pooltypeid2와 pooltypeid10과 동일한 값, 분포)

# 전처리순서 
yard > garage > pool 

1. yardbuilding17 의 NaN을 0으로 채운다
3. garagecarcnt의 NaN을 0으로 채운다ㅏ
    2. garagecarcnt가 null인데 garagetotalsqft 이 126.0인 값을 1로 채운다
    2. garagecarcnt의 NaN을 0으로 채운다ㅏ 
4. garagetotalsqft을 garagecarcnt에 따른 mean값을 채운다
5. poolcnt의 nan을 0값으로 채운다
5. pooltypeid2,7,10 의 nan을 0값으로 채운다
6. poolsizesum 을 추측하는 garagetotalsqft, finishedfloor1sqarefeet, poolcnt에 의한 StatsModel로 nan값을 predict해 채운다


1.yardbuilding17 의 NaN을 0으로 채운다

In [35]:
train2016.loc[train2016.yardbuildingsqft17.isnull(), 'yardbuildingsqft17'] = 0.0

2.garagecarcnt의 NaN을 0으로 채운다ㅏ
    1. garagecarcnt가 null인데 garagetotalsqft 이 126.0인 값을 1로 채운다
    2. garagecarcnt의 NaN을 0으로 채운다ㅏ 

In [36]:
train2016[['garagecarcnt', 'garagetotalsqft']][(train2016.garagetotalsqft != 0) & (train2016.garagecarcnt == 0)]

,garagecarcnt,garagetotalsqft
57609,0.0,126.0


In [46]:
train2016.loc[57609, 'garagecarcnt'] = 1.0

3.garagetotalsqft을 garagecarcnt에 따른 mean값을 채운다

In [38]:
mean_garagesqft = train2016[prop_garages].groupby(['garagecarcnt']).agg('mean')
mean_garagesqft

,garagetotalsqft
garagecarcnt,
0.0,0.434483
1.0,125.248989
2.0,390.600082
3.0,758.722474
4.0,969.465649
5.0,1304.351852
6.0,1645.812500
7.0,1982.363636
8.0,2313.000000


In [55]:
def garagesqft_fillna(garagecarcnt):
    x = round(mean_garagesqft.loc[garagecarcnt][0], 0)
    print(x)
    train2016.loc[(train2016.garagecarcnt == garagecarcnt) & (train2016.garagetotalsqft == 0),'garagetotalsqft'] = x

In [56]:
for i in mean_garagesqft.index[1:] :
    garagesqft_fillna(i)

125.0
391.0
759.0
969.0
1304.0
1646.0
1982.0
2313.0
2704.0
3077.0
3355.0
4048.0
4384.0
7339.0


4.poolcnt의 nan을 0값으로 채운다

In [41]:
train2016.poolcnt.fillna(value=0.0, inplace=True)

5.pooltypeid2,7,10의 nan을 0으로 채운다

In [66]:
train2016[['pooltypeid2','pooltypeid7','pooltypeid10']] = train2016[['pooltypeid2','pooltypeid7','pooltypeid10']].fillna(value=0)

6.poolsizesum 을 추측하는 garagetotalsqft, finishedsquarefeet50, poolcnt에 의한 StatsModel로 nan값을 predict해 채운다


(poolsizesum와 상관이 있는 면적 데이터로 - garagetotalsqft, finishedsquarefeet50- 와 poolcnt)

In [42]:
model_poolsizesum = sm.OLS.from_formula("poolsizesum~I(finishedsquarefeet50*0.01)+I(garagetotalsqft*0.01)+C(poolcnt)+0", data = train2016)
poolsizesum_result = model_poolsizesum.fit()
predicted_poolsizesum = poolsizesum_result.predict(train2016[['poolcnt','finishedsquarefeet50', 'garagetotalsqft']].dropna())

In [43]:
for i in predicted_poolsizesum.index :
    if train2016.loc[i, 'poolcnt'] > 0:
        train2016.loc[i, 'poolsizesum'] = round(predicted_poolsizesum.loc(0)[i], 0)
    else:
        pass


In [51]:
len(train2016[(train2016.poolcnt !=0) & (train2016.poolsizesum !=0)])

17901